### Smoking and Drinking Dataset with body signal

#### This dataset is collected from National Health Insurance Service in Korea. All personal information and sensitive data were excluded.
#### The purpose of this dataset is to:

##### 1- Analysis of body signal
##### 2- Classification of smoker or drinker

### Importing libraries 

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os
print("Tensor Flow Version: {}".format(tf.__version__))


Tensor Flow Version: 2.15.0


### Load the Dataset


In [4]:
path="C:\\Users\\mybox\\Downloads\\smoking.csv"
filename_read=os.path.join(path,"C:\\Users\\mybox\\Downloads\\smoking.csv")
df = pd.read_csv(filename_read, na_values=['NA', '?'])

### Check if there is misssing data

In [ ]:
missing_values = df.isnull().sum()

print("Missing Values:\n", missing_values)

if missing_values.sum() == 0:
    print("No missing values in the DataFrame.")
else:
    print("There are missing values in the DataFrame.")